# factory-planning-i.py

notebook 版本

## Factory Planning I Example
Source: http://www.gurobi.com/resources/examples/factory-planning-I

version 1

### Step 1: Import functions and create Constants

In [1]:
from gurobipy import *


In [2]:
# tested with Python 3.5.2 & Gurobi 7.0.1

products = ["Prod1", "Prod2", "Prod3", "Prod4", "Prod5", "Prod6", "Prod7"]
machines = ["grinder", "vertDrill", "horiDrill", "borer", "planer"]
time_periods = ["January", "February", "March", "April", "May", "June"]

# 获利表
profit_contribution = {"Prod1":10, "Prod2":6, "Prod3":8, "Prod4":4, "Prod5":11, "Prod6":9, "Prod7":3}

# 生产用时表(为何不以  Prod 为坐标)
time_table = {
    "grinder": {    "Prod1": 0.5, "Prod2": 0.7, "Prod5": 0.3,
                    "Prod6": 0.2, "Prod7": 0.5 },
    "vertDrill": {  "Prod1": 0.1, "Prod2": 0.2, "Prod4": 0.3,
                    "Prod6": 0.6 },
    "horiDrill": {  "Prod1": 0.2, "Prod3": 0.8, "Prod7": 0.6 },
    "borer": {      "Prod1": 0.05,"Prod2": 0.03,"Prod4": 0.07,
                    "Prod5": 0.1, "Prod7": 0.08 },
    "planer": {     "Prod3": 0.01,"Prod5": 0.05,"Prod7": 0.05 }
}


# number of machines down
down = {("January","grinder"): 1, ("February", "horiDrill"): 2, ("March", "borer"): 1,
        ("April", "vertDrill"): 1, ("May", "grinder"): 1, ("May", "vertDrill"): 1,
        ("June", "planer"): 1, ("June", "horiDrill"): 1}

# number of each machine available
qMachine = {"grinder":4, "vertDrill":2, "horiDrill":3, "borer":1, "planer":1} 

# market limitation of sells
upper = {
    ("January", "Prod1") : 500,
    ("January", "Prod2") : 1000,
    ("January", "Prod3") : 300,
    ("January", "Prod4") : 300,
    ("January", "Prod5") : 800,
    ("January", "Prod6") : 200,
    ("January", "Prod7") : 100,
    ("February", "Prod1") : 600,
    ("February", "Prod2") : 500,
    ("February", "Prod3") : 200,
    ("February", "Prod4") : 0,
    ("February", "Prod5") : 400,
    ("February", "Prod6") : 300,
    ("February", "Prod7") : 150,
    ("March", "Prod1") : 300,
    ("March", "Prod2") : 600,
    ("March", "Prod3") : 0,
    ("March", "Prod4") : 0,
    ("March", "Prod5") : 500,
    ("March", "Prod6") : 400,
    ("March", "Prod7") : 100,
    ("April", "Prod1") : 200,
    ("April", "Prod2") : 300,
    ("April", "Prod3") : 400,
    ("April", "Prod4") : 500,
    ("April", "Prod5") : 200,
    ("April", "Prod6") : 0,
    ("April", "Prod7") : 100,
    ("May", "Prod1") : 0,
    ("May", "Prod2") : 100,
    ("May", "Prod3") : 500,
    ("May", "Prod4") : 100,
    ("May", "Prod5") : 1000,
    ("May", "Prod6") : 300,
    ("May", "Prod7") : 0,
    ("June", "Prod1") : 500,
    ("June", "Prod2") : 500,
    ("June", "Prod3") : 100,
    ("June", "Prod4") : 300,
    ("June", "Prod5") : 1100,
    ("June", "Prod6") : 500,
    ("June", "Prod7") : 60,
}



### Step 2: Create  model

In [3]:
storeCost = 0.5
storeCapacity = 100
endStock = 50
hoursPerMonth = 2*8*24

model = Model('Factory Planning I')



### Step 3: Create activitiy variables

In [4]:
manu = model.addVars(time_periods, products, vtype=GRB.CONTINUOUS, name="Manu")
held = model.addVars(time_periods, products , ub=storeCapacity, vtype=GRB.CONTINUOUS, name="held")
sell = model.addVars(time_periods, products, ub=upper, name="Sell") # quantity

# machine_hours = model.addVars(time_periods, machines, vtype=GRB.INTEGER, name="machine_hours")
# machine_num = model.addVars(time_periods, machines, vtype=GRB.INTEGER, name="machine_num")

### Step 4: Set objective function

In [5]:
# Objective
obj = quicksum(profit_contribution[product] * sell[time_period, product] -  storeCost * held[time_period, product]  for time_period in time_periods    
                    for product in products)

model.setObjective(obj, GRB.MAXIMIZE)

### Step 5: Add constraints

In [6]:
# 这些源代码默认的约束条件改在 在下一格自助添加

# 初始,每月以及最终库存
# Initial Balance
# model.addConstrs((manu[time_periods[0], product] == sell[time_periods[0], product] 
#                   + held[time_periods[0], product] for product in products), name="Initial_Balance")
    
#Balance
# model.addConstrs((held[time_periods[time_periods.index(time_period) -1], product] + manu[time_period, product] ==
#                     sell[time_period, product] + held[time_period, product] 
#                     for product in products for time_period in time_periods if time_period != time_periods[0]), name="Balance")

# End store
# model.addConstrs((held[time_periods[-1], product] == endStock for product in products),  name="End_Balance")

# 设备 down 的时间约束
# Capacity
# model.addConstrs((quicksum(time_table[machine][product] * manu[time_period, product] for product in time_table[machine])
#                            <= hoursPerMonth * (qMachine[machine] - down[time_period, machine]) 
#                             for machine in machines for time_period in time_periods if (time_period, machine) in down), name = "Capacity")

# 设备 不 down 的时间约束
# model.addConstrs((quicksum(time_table[machine][product] * manu[time_period, product] for product in time_table[machine])
#                             <= hoursPerMonth * qMachine[machine] 
#                             for machine in machines for time_period in time_periods if (time_period, machine) not in down), name = "Capacity")


### Step 6:  Add Our Constrains & Solve model

In [7]:
## 3 个库存条件; 条件2采用了类似差分方程的写法
# At the start of January there is no product inventory
# Init + Manu = Sell + Hold 
# 注意:如果使用错 addConstr 而不是 addConstr[s]
# 会报错误: TypeError: unsupported operand type(s) for -: 'generator' and 'NoneType'
#  或是 NameError: name 'p' is not defined
# init month
model.addConstrs( (manu[time_periods[0],p] == sell[time_periods[0],p] 
                  + held[time_periods[0],p] for p in products), name="held_init_Jan_")

# Hold[-1] + Manu = Sell + Hold 
model.addConstrs( (held[time_periods[time_periods.index(t)-1],p] + manu[t,p] == sell[t,p] 
                  + held[t,p] for p in products for t in time_periods if t != time_periods[0]), name="held_init_Jan_")

# last month:  by the end of June there should be 50 units of each product in inventory.
model.addConstrs( (held[time_periods[-1],p] == endStock for p in products), name="held_last_month_")

## 两个设备时间约束; 采用了 线性相乘取和的写法  
# time_table[m] = {    "Prod1": 0.5, "Prod2": 0.7, "Prod5": 0.3, "Prod6": 0.2, "Prod7": 0.5 }
# manu[t,p] = "Prod1": 20 , ...
# quicksum( manu[t,p] * time_table[m][p] )
# 设备 down 的时间约束
# # Capacity
model.addConstrs(  (quicksum( manu[t,p] * time_table[m][p] for p in time_table[m] )
                    <= (qMachine[m]-down[t,m]) * hoursPerMonth for t in time_periods for m in machines 
                             if (t,m) in down), name="machine_not_down") 

# 设备 不 down 的时间约束: (这个约束条件是可以不加的, 最优值一样)
# 比 for p in products if time_table[m].has_key(p))
# 更好的写法: for product in time_table[m]
# model.addConstrs(  (quicksum(( manu[t,p] * time_table[m][p] for p in products if time_table[m].has_key(p)))
#                     <= qMachine[m] * hoursPerMonth for t in time_periods for m in machines 
#                     if (t,m) not in down), name="machine_not_down") 

model.optimize()

print "最佳值应为: # obj = Best objective 9.371517857e+04"
if int(model.ObjVal) == int(9.371517857e+04):
    print "\n最佳值 {} 与答案一致.".format(model.ObjVal)
else:
    print "\n而不是 {} ....".format(model.ObjVal)


Optimize a model with 57 rows, 126 columns and 200 nonzeros
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [5e-01, 1e+01]
  Bounds range     [6e+01, 1e+03]
  RHS range        [5e+01, 1e+03]
Presolve removed 51 rows and 109 columns
Presolve time: 0.01s
Presolved: 6 rows, 17 columns, 23 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.4476010e+04   2.241010e+02   0.000000e+00      0s
       3    9.3715179e+04   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.02 seconds
Optimal objective  9.371517857e+04
最佳值应为: # obj = Best objective 9.371517857e+04

最佳值 93715.1785714 与答案一致.


### Step 8: Print variable values for optimal solution

In [8]:
# Display solution (print the name of each variable and the solution value)

def printAllVars():
    """
    打印所有变量
    """
    for v in model.getVars():
        if v.X != 0:
            print("%s %f" % (v.Varname, v.X))
# printAllVars()
print "\tB:  ",
for w in time_periods:
    print " [{0}]".format(w),
print 
for p in products:
    print "\t[A_{0}]".format(p), 
    # print('Plant %s open' % p)
    for w in time_periods:
        #if transport[w,p].x > 0:
        #    print('  Transport %g units to warehouse %s' % \
        #          (transport[w,p].x, w))
        print "\t{}".format(int(manu[w, p].x)), 
    print
        


	B:    [January]  [February]  [March]  [April]  [May]  [June]
	[A_Prod1] 	500 	700 	0 	200 	0 	550
	[A_Prod2] 	888 	600 	0 	300 	100 	550
	[A_Prod3] 	382 	117 	0 	400 	600 	0
	[A_Prod4] 	300 	0 	0 	500 	100 	350
	[A_Prod5] 	800 	500 	0 	200 	1100 	0
	[A_Prod6] 	200 	300 	400 	0 	300 	550
	[A_Prod7] 	0 	250 	0 	100 	100 	0
